In [29]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb

import concurrent.futures, json, os, re, requests, subprocess

if not os.path.exists('python-utils'):
    subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
notebook_wide_display()

In [30]:
import os, psutil, subprocess

In [31]:
def error(msg):
    global errors
    Stat.log('ThumbnailWorker', 'critical', msg)
    errors.append(msg)
    
def reportStatus():
    if len(errors):
        Stat.log('ThumbnailWorker', 'down', 'Failed %d health checks' % len(errors), details=(' | '.join(errors)))
    else:
        Stat.log('ThumbnailWorker', 'up', 'Passed all health checks')
        
def warning(msg):
    Stat.log('ThumbnailWorker', 'warning', msg)
    
def clearErrors():
    global errors
    errors = []

loadAvgMax = 50

def checkLoadAvgMax():
    (load1m, load5m, load15m) = os.getloadavg()
    if load5m > loadAvgMax:
        error('Load avg %.1f (test > %.1f)' % (load5m, loadAvgMax))

cpuMax = 90 # % of all CPU (i.e. 100=all cores at full utilization)

def checkCPUMax():
    cpu = psutil.cpu_percent(3)
    if cpu > cpuMax:
        error('CPU usage %d%% (test > %d%%)' % (cpu, cpuMax))        
        
maxDiskUsage = 95

def checkDiskUsage():
    dfCommand = 'df -l --print-type --exclude-type=tmpfs --exclude-type=devtmpfs'
    filesystems = subprocess.check_output(dfCommand, shell=True).decode('utf8').split('\n')[1:]

    for filesystem in filesystems:
        fields = filesystem.split()
        if not len(fields):
            continue
        diskUsage = int(fields[-2][:-1])
        if diskUsage > maxDiskUsage:
            mountPoint = fields[-1]
            error('%s is %d%% full (test > %d%%)' % (mountPoint, diskUsage, maxDiskUsage))

In [32]:
maxChromedriverCount = 6

def checkChromedriver(p):
    if p.ppid() == 1:
        warning('Killing chromedriver pid=%d because process has no parent' % p.pid)
        try:
            p.kill()
        except psutil.AccessDenied:
            error('Insufficient permission to kill chromdriver pid=%d' % p.pid)
    else:
        parentName = psutil.Process(p.ppid()).name()
        if parentName != 'ruby':
            error('chromedriver pid=%d has unknown parent process pid=%d name=%s' % (p.pid, p.ppid(), parentName))

def checkChromedrivers():
    for p in psutil.process_iter():
        if p.name() == 'chromedriver':
            checkChromedriver(p)
            
    time.sleep(5) # wait for killed processes (if any) to die
    
    chromedriverCount = 0
    for p in psutil.process_iter():
        if p.name() == 'chromedriver':
            chromedriverCount += 1
    
    if chromedriverCount > maxChromedriverCount:
        error('%d chromedrivers running (test > %d)' % (chromedriverCount, maxChromedriverCount))


In [33]:
def checkChrome(p):
    if p.ppid() == 1 and '--headless' in ' '.join(p.cmdline()):
        warning('Killing headless chrome pid=%d because process has no parent' % p.pid)
        try:
            p.kill()
        except psutil.AccessDenied:
            error('Insufficient permission to kill chrome pid=%d' % p.pid)

def checkChromes():
    for p in psutil.process_iter():
        if p.name() == 'chrome':
            checkChrome(p)

In [34]:
def checkAll():
    clearErrors()
    checkLoadAvgMax()
    checkCPUMax()
    checkDiskUsage()
    checkChromedrivers()
    checkChromes()
    reportStatus()

In [27]:
while True:
    sleep_until_next_period(60)
    checkAll()    

sleep_until_next_period(60, 0) sleeping 55 seconds until 19:37:00


KeyboardInterrupt: 